### Importing packages

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
#p = Augmentor.Pipeline("train/not_empty")
#p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
#p.sample(52)

###  Load the pretrained Network

In [3]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model_vgg16_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

###  Freeze the layers

In [5]:
for layer in model_vgg16_conv.layers:
    layer.trainable = False

###  Training parameters

In [6]:
img_width, img_height = 150, 150
train_data_dir = 'train2' #40 images from each class for training
model_weights_file = 'vgg16-xfer-weights.h5'
nb_train_samples = 80
nb_val_samples = 4
nb_epochs = 20

###  Build a classification model on top of Base Network

In [7]:
input = Input(shape=(img_width, img_height, 3))
output_vgg16_conv = model_vgg16_conv(input)
x = Flatten()(output_vgg16_conv)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(2, activation='softmax')(x)
model = Model(input=input, output=x)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                524352    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 15,243,330
Trainable params: 528,642
Non-trainable params: 14,714,688
__________________________________________________________

###  Dataset Preparation

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=2, class_mode='categorical')

Found 80 images belonging to 2 classes.


###  Training

In [10]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='acc', mode = 'max', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks, samples_per_epoch=nb_train_samples, 
                              nb_epoch=nb_epochs, nb_val_samples=nb_val_samples)

print('Training Completed!')

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
40/40 [==============================] - 34s 851ms/step - loss: 0.8841 - acc: 0.5250
Epoch 2/20
40/40 [==============================] - 35s 882ms/step - loss: 0.6331 - acc: 0.7000
Epoch 3/20
40/40 [==============================] - 36s 912ms/step - loss: 0.5403 - acc: 0.7875
Epoch 4/20
40/40 [==============================] - 34s 850ms/step - loss: 0.4110 - acc: 0.8250
Epoch 5/20
40/40 [==============================] - 34s 850ms/step - loss: 0.4155 - acc: 0.8500
Epoch 6/20
40/40 [==============================] - 36s 893ms/step - loss: 0.3156 - acc: 0.8750
Epoch 7/20
40/40 [==============================] - 41s 1s/step - loss: 0.2520 - acc: 0.9125
Epoch 8/20
40/40 [==============================] - 37s 937ms/step - loss: 0.2849 - acc: 0.8750
Epoch 9/20
40/40 [==============================] - 38s 953ms/step - loss: 0.3387 - acc: 0.9250
Epoch 10/20
40/40 [==============================] - 36s 910ms/step - loss: 0.2223 - acc: 0

###  Test the model

In [12]:
import glob
correct = 0
total = 0

#test folder contains 13 images from each class 

label = ['empty', 'not_empty']

for filename in glob.glob('/home/divyaansh/Downloads/OneDrive-2019-10-10/test/empty/*.jpg'):
    img = image.load_img(filename, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    ind = np.where(features >= 0.5)[1][0]
    if(label[ind] == 'empty'):
        correct+=1
    total+=1

for filename in glob.glob('/home/divyaansh/Downloads/OneDrive-2019-10-10/test/not_empty/*.jpg'):
    img = image.load_img(filename, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    ind = np.where(features >= 0.5)[1][0]
    if(label[ind] == 'not_empty'):
        correct+=1
    total+=1    
print("Accuracy:",correct/total);

Accuracy: 0.8076923076923077


In [20]:
from joblib import dump, load
dump(model,'vgg16_model.joblib')

['vgg16_model.joblib']

In [ ]:
#from PIL import Image
#import PIL.Image

#import pytesseract
#from pytesseract import image_to_string
##pytesseract.pytesseract.tesseract_cmd = 'sudo /usr/local/lib/python3.6/dist-packages'
##print image_to_string(Image.open('empty.jpg'))
#image_to_string(Image.open('img.jpg'))